## [Python实战异步爬虫(协程)+分布式爬虫(多进程)](https://blog.csdn.net/SL_World/article/details/86633611)

在讲解之前，我们先来通过一幅图看清多进程和协程的爬虫之间的原理及其区别。(图片来源于网络)

![aiohttp](aiohttp.png)

这里，异步爬虫不同于多进程爬虫，它使用单线程(即仅创建一个事件循环，然后把所有任务添加到事件循环中)就能并发处理多任务。在轮询到某个任务后，当遇到耗时操作(如请求URL)时，挂起该任务并进行下一个任务，当之前被挂起的任务更新了状态(如获得了网页响应)，则被唤醒，程序继续从上次挂起的地方运行下去。极大的减少了中间不必要的等待时间。

对于协程(Asyncio库)的原理及实现请见：[Python异步IO之协程(详解)](https://blog.csdn.net/SL_World/article/details/86597738)
对于多进程的知识讲解及实现请见：[廖雪峰-Python多进程](https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001431927781401bb47ccf187b24c3b955157bb12c5882d000)
在有了Asyncio异步IO库实现协程后，我们还需要实现异步网页请求。因此，aiohttp库应运而生。

### 使用aiohttp库实现异步网页请求

在我们写普通的爬虫程序时，经常会用到requests库用以请求网页并获得服务器响应。而在协程中，由于requests库提供的相关方法不是可等待对象(awaitable),使得无法放在await后面，因此无法使用requests库在协程程序中实现请求。

在此，官方专门提供了一个aiohttp库，用来实现异步网页请求等功能，简直就是异步版的requests库，当然需要我们手动安装该库

【基础实现】：在官方文档中，推荐使用ClientSession()函数来调用网页请求等相关方法。

- 首先，我们需要引入aiohttp模块。
- 然后，我们在协程中使用ClientSession()的get()或request()方法来请求网页。(其中async with是异步上下文管理器，其封装了异步实现等功能)


In [ ]:
import aiohttp

async with aiohttp.ClientSession() as session:
    async with session.get('http://httpbin.org/get') as resp:
        print(resp.status)
        print(await resp.text())

###  测试普通爬虫程序

下面是一个普通的同步代码，实现顺序爬取10个URL的title。

In [6]:
import time
from lxml import etree
import requests

urls = [
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16488',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16583',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16380',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16911',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16581',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16674',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16112',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/17343',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16659',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16449',
]

def get_title(url, cnt):
    response = requests.get(url)
    html = response.content
    title = etree.HTML(html).xpath('//*[@id="title"]/text()')
    print('第 {} 个 title: {}'.format(cnt,''.join(title)))
    
if __name__ == '__main__':
    start = time.time()
    i = 0
    for url in urls:
        i += 1
        start_each = time.time()
        get_title(url,i)
        print('第 {} 个title 爬取耗时：{}'.format(i,float(time.time()-start_each)))
    print('爬取总耗时：{}秒'.format(float(time.time()-start)))

第 1 个 title: Norm Conflict Resolution in Stochastic Domains
第 1 个title 爬取耗时：1.2110004425048828
第 2 个 title: Algorithms for Trip-Vehicle Assignment in Ride-Sharing
第 2 个title 爬取耗时：1.9450290203094482
第 3 个 title: Tensorized Projection for High-Dimensional Binary Embedding
第 3 个title 爬取耗时：2.9749748706817627
第 4 个 title: Synthesis of Programs from Multimodal Datasets
第 4 个title 爬取耗时：1.2339951992034912
第 5 个 title: Video Summarization via Semantic Attended Networks
第 5 个title 爬取耗时：1.491034746170044
第 6 个 title: TIMERS: Error-Bounded SVD Restart on Dynamic Networks
第 6 个title 爬取耗时：2.0010170936584473
第 7 个 title: Memory Management With Explicit Time in Resource-Bounded Agents
第 7 个title 爬取耗时：3.511124849319458
第 8 个 title: Mitigating Overexposure in Viral Marketing
第 8 个title 爬取耗时：1.5009915828704834
第 9 个 title: Neural Link Prediction over Aligned Networks
第 9 个title 爬取耗时：2.6780362129211426
第 10 个 title: Dual Deep Neural Networks Cross-Modal Hashing
第 10 个title 爬取耗时：5.672969102859497
爬取总耗时：24.

可见，平均每请求完一个URL并解析该HTML耗时2.6秒左右。本次程序运行总耗时24.3秒。

### 测试基于协程的异步爬虫程序

下面，是使用了协程的异步爬虫程序。etree模块用于解析HTML，aiohttp是一个利用asyncio的库，它的API看起来很像请求的API，可以暂时看成协程版的requests。

In [ ]:
import time
from lxml import etree
import aiohttp,asyncio

urls = [
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16488',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16583',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16380',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16911',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16581',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16674',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16112',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/17343',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16659',
    'https://aaai.org/ocs/index.php/AAAI/AAAI18/paper/viewPaper/16449',
]

titles = []
sem = asyncio.semaphore(10)  # 信号量，控制协程数，防止爬的过快

async def get_title(url):
    


def get_title(url, cnt):
    response = requests.get(url)
    html = response.content
    title = etree.HTML(html).xpath('//*[@id="title"]/text()')
    print('第 {} 个 title: {}'.format(cnt,''.join(title)))
    
if __name__ == '__main__':
    start = time.time()
    i = 0
    for url in urls:
        i += 1
        start_each = time.time()
        get_title(url,i)
        print('第 {} 个title 爬取耗时：{}'.format(i,float(time.time()-start_each)))
    print('爬取总耗时：{}秒'.format(float(time.time()-start)))